# Team Operations : Reset, Stop, Resume and Abort

In [3]:
import asyncio
from autogen_ext.models.openai import OpenAIChatCompletionClient,AzureOpenAIChatCompletionClient
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv('AZURE_OPENAI_API_KEY')
model_client = AzureOpenAIChatCompletionClient(
    # azure_deployment="Basavaraj-Bijali",  # This is your deployment name, not the model name
    model="gpt-4o",  # Model name, e.g., "gpt-4"
    api_version="2024-12-01-preview",  # Use the API version your deployment supports
    azure_endpoint="https://tvscs-az-openai.openai.azure.com/",
    api_key=api_key
)

In [4]:
from autogen_agentchat.agents import AssistantAgent
add_1_agent_first = AssistantAgent(
    name = 'add_1_agent_first',
    model_client=model_client,
    system_message="Add 1 to the number, first number is 0. Give result as output"
)

add_1_agent_second = AssistantAgent(
    name = 'add_1_agent_second',
    model_client=model_client,
    system_message="Add 1 to the number. Give result as output."
)
 
add_1_agent_third = AssistantAgent(
    name = 'add_1_agent_third',
    model_client=model_client,
    system_message="Add 1 to the number. Give result as output."
)


In [5]:
from autogen_agentchat.teams import RoundRobinGroupChat

team = RoundRobinGroupChat(
    [add_1_agent_first, add_1_agent_second, add_1_agent_third],
    max_turns=3
)

In [6]:
from autogen_agentchat.ui import Console

await Console(team.run_stream())

---------- TextMessage (add_1_agent_first) ----------
1


/Users/basavarajbijali/Desktop/Personal/Udemy_AI_AGENT_COURCE_Jun25/Autogen/autogen_env/lib/python3.12/site-packages/autogen_agentchat/agents/_assistant_agent.py:955: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-11-20. Model mapping in autogen_ext.models.openai may be incorrect. Set the model to gpt-4o-2024-11-20 to enhance token/cost estimation and suppress this warning.
  model_result = await model_client.create(


---------- TextMessage (add_1_agent_second) ----------
2
---------- TextMessage (add_1_agent_third) ----------
3


TaskResult(messages=[TextMessage(source='add_1_agent_first', models_usage=RequestUsage(prompt_tokens=24, completion_tokens=2), metadata={}, content='1', type='TextMessage'), TextMessage(source='add_1_agent_second', models_usage=RequestUsage(prompt_tokens=29, completion_tokens=2), metadata={}, content='2', type='TextMessage'), TextMessage(source='add_1_agent_third', models_usage=RequestUsage(prompt_tokens=39, completion_tokens=2), metadata={}, content='3', type='TextMessage')], stop_reason='Maximum number of turns 3 reached.')

# Resuming a Team
Teams are stateful and maintains the conversation history and context after each run, unless you reset the team.


We can resume a team to continue from where it left off by calling the run() or run_stream() method without a **new task**


In [7]:
await Console(team.run_stream())

---------- TextMessage (add_1_agent_first) ----------
4
---------- TextMessage (add_1_agent_second) ----------
5
---------- TextMessage (add_1_agent_third) ----------
6


TaskResult(messages=[TextMessage(source='add_1_agent_first', models_usage=RequestUsage(prompt_tokens=50, completion_tokens=2), metadata={}, content='4', type='TextMessage'), TextMessage(source='add_1_agent_second', models_usage=RequestUsage(prompt_tokens=55, completion_tokens=2), metadata={}, content='5', type='TextMessage'), TextMessage(source='add_1_agent_third', models_usage=RequestUsage(prompt_tokens=64, completion_tokens=2), metadata={}, content='6', type='TextMessage')], stop_reason='Maximum number of turns 3 reached.')

In [8]:
await Console(team.run_stream())

---------- TextMessage (add_1_agent_first) ----------
7
---------- TextMessage (add_1_agent_second) ----------
8
---------- TextMessage (add_1_agent_third) ----------
9


TaskResult(messages=[TextMessage(source='add_1_agent_first', models_usage=RequestUsage(prompt_tokens=76, completion_tokens=2), metadata={}, content='7', type='TextMessage'), TextMessage(source='add_1_agent_second', models_usage=RequestUsage(prompt_tokens=81, completion_tokens=2), metadata={}, content='8', type='TextMessage'), TextMessage(source='add_1_agent_third', models_usage=RequestUsage(prompt_tokens=89, completion_tokens=2), metadata={}, content='9', type='TextMessage')], stop_reason='Maximum number of turns 3 reached.')

# team resumed from where it left off in the output above, and the first message is from the next agent after the last agent that spoke before the team stopped.

In [9]:
await Console(team.run_stream(task = 'What was the largest number you got in the result?'))

---------- TextMessage (user) ----------
What was the largest number you got in the result?
---------- TextMessage (add_1_agent_first) ----------
The largest number I got in the result was **9**.
---------- TextMessage (add_1_agent_second) ----------
That’s correct! The largest number so far is **9**. :)
---------- TextMessage (add_1_agent_third) ----------
You both are absolutely right! The largest number so far is **9**. 😊


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='What was the largest number you got in the result?', type='TextMessage'), TextMessage(source='add_1_agent_first', models_usage=RequestUsage(prompt_tokens=118, completion_tokens=14), metadata={}, content='The largest number I got in the result was **9**.', type='TextMessage'), TextMessage(source='add_1_agent_second', models_usage=RequestUsage(prompt_tokens=135, completion_tokens=16), metadata={}, content='That’s correct! The largest number so far is **9**. :)', type='TextMessage'), TextMessage(source='add_1_agent_third', models_usage=RequestUsage(prompt_tokens=156, completion_tokens=18), metadata={}, content='You both are absolutely right! The largest number so far is **9**. 😊', type='TextMessage')], stop_reason='Maximum number of turns 3 reached.')

# Reset our Team

In [10]:
await team.reset() # on_reset() on all agents

In [11]:
await Console(team.run_stream())

---------- TextMessage (add_1_agent_first) ----------
1
---------- TextMessage (add_1_agent_second) ----------
2
---------- TextMessage (add_1_agent_third) ----------
3


TaskResult(messages=[TextMessage(source='add_1_agent_first', models_usage=RequestUsage(prompt_tokens=24, completion_tokens=2), metadata={}, content='1', type='TextMessage'), TextMessage(source='add_1_agent_second', models_usage=RequestUsage(prompt_tokens=29, completion_tokens=2), metadata={}, content='2', type='TextMessage'), TextMessage(source='add_1_agent_third', models_usage=RequestUsage(prompt_tokens=39, completion_tokens=2), metadata={}, content='3', type='TextMessage')], stop_reason='Maximum number of turns 3 reached.')

## Covered in Future Videos in the Module

# Aborting a Team

Different from stopping a team, aborting a team will immediately stop the team and raise a CancelledError exception.

In [ ]:
from autogen_core import CancellationToken

cancellation_token = CancellationToken()

run2 = asyncio.create_task(
    Console(team.run_stream(task = 'Give a short Story about a lion atmost 40 words',cancellation_token=cancellation_token))
)

await asyncio.sleep(2)
cancellation_token.cancel()

try:
    result = await run2
except asyncio.CancelledError():
    print("Task Was Cancelled")

TypeError: catching classes that do not inherit from BaseException is not allowed